## Задание
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
* Наименование вакансии.
* Предлагаемую зарплату (отдельно минимальную, максимальную и валюту).
* Ссылку на саму вакансию.
* Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [2]:
def check_vacancy(job, city):
    main_link = 'https://hh.ru'
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) \
        Chrome/89.0.4389.90 Safari/537.36'}
    link = f'{main_link}/search/vacancy/'
    params = {'clusters': 'true', 'enable_snippets': 'true', 'text': {job}, 'L_save_area': 'true', 'area': {city},
              'showClusters': 'true'}

    result = []
    while True:
        response = requests.get(link, headers=header, params=params)
        soup = bs(response.text, 'html.parser')

        if response.ok:
            vacancy_list = soup.find_all('div', {'class': 'vacancy-serp-item'})
            for i in vacancy_list:
                data = {}

                # title
                vacancy_title = i.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
                title = vacancy_title.text
                data['title'] = title

                # vacancy url_link
                data['link'] = vacancy_title['href']

                # city
                data['city'] = i.find('span', {'data-qa': 'vacancy-serp__vacancy-address'}).text

                # company name
                company_name = i.find('div', {'class': 'vacancy-serp-item__meta-info'}).find('a').getText() \
                    .replace(u'\xa0', u'')
                data['company_name'] = company_name

                # metro
                metro_station = i.find('span', {'class': 'vacancy-serp-item__meta-info'}).findChild()
                if not metro_station:
                    metro_station = None
                else:
                    metro_station = metro_station.getText()
                data['metro_station'] = metro_station

                # salary
                vacancy_salary = i.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
                vacancy_min = None
                vacancy_max = None
                vacancy_currency = None

                if vacancy_salary is not None:
                    vacancy_salary = vacancy_salary.getText().replace('\u202f', '')
                    vacancy_currency = vacancy_salary.split()[-1]

                    if vacancy_salary.find('–') > -1:
                        vacancy_min = int(vacancy_salary.split()[0])
                        vacancy_max = int(vacancy_salary.split()[2])
                    elif vacancy_salary.find('от'):
                        vacancy_min = int(vacancy_salary.split()[1])
                    elif vacancy_salary.find('до'):
                        vacancy_max = int(vacancy_salary.split()[1])

                data['salary_min'] = vacancy_min
                data['salary_max'] = vacancy_max
                data['salary_currency'] = vacancy_currency

                result.append(data)
                next_page = soup.find('a', {'data-qa': 'pager-next'})

            if next_page is not None:
                pass
            else:
                break
            params = next_page['href']

    return result

In [3]:
def parser_vacancy(job, city):       
    v = []
    v.extend(check_vacancy(job, city))
    
    df = pd.DataFrame(v)

    return df

In [4]:
def vacancy_count(job, city):
    v = []
    v.extend(check_vacancy(job, city))
    
    df = pd.DataFrame(v)

    return f'Количество вакансий {job} - {df.shape[0]}'

In [5]:
vacancy_count('product manager', 1)

'Количество вакансий product manager - 2000'

In [6]:
vacancy_count('product owner', 1)

'Количество вакансий product owner - 591'

In [7]:
vacancy_count('product analyst', 1)

'Количество вакансий product analyst - 1023'

In [8]:
vacancy_count('data analyst', 1)

'Количество вакансий data analyst - 1625'

In [9]:
vacancy_count('data scientist', 1)

'Количество вакансий data scientist - 348'

In [10]:
df = parser_vacancy('product analyst', 1)
df

,title,link,city,company_name,metro_station,salary_min,salary_max,salary_currency
0,Lead Product Manager,https://hh.ru/vacancy/43394185,Москва,Scalable Solutions,None,300000.0,450000.0,руб.
1,Senior Product Analyst,https://hh.ru/vacancy/40375714,Москва,FunCorp,None,250000.0,350000.0,руб.
2,Product Manager,https://hh.ru/vacancy/42277954,Москва,Scalable Solutions,None,200000.0,300000.0,руб.
3,Product manager,https://hh.ru/vacancy/43704280,Москва,Silverbird,None,200000.0,320000.0,руб.
4,Product Manager (CryptoProcessing),https://hh.ru/vacancy/43908772,Москва,Unlimint,None,NaN,250000.0,руб.
...,...,...,...,...,...,...,...,...
1018,Developer (JS),https://hh.ru/vacancy/42783578,Москва,CUSTOMERTIMES,None,NaN,NaN,None
1019,Technical Lead в Data Science-команду/ML Engineer,https://hh.ru/vacancy/42775724,Москва,"«РОСБАНК, Societe Generale Group», с опытом ра...",None,NaN,NaN,None
1020,Head of User Acquisition / Руководитель группы...,https://hh.ru/vacancy/43180207,Москва,ProductStar,None,150000.0,200000.0,руб.
1021,Специалист поддержки автоматизированной банков...,https://hh.ru/vacancy/43166785,Москва,Cornerstone Russia,None,NaN,NaN,None


In [11]:
# df.to_excel("jobs.xlsx")